In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import datasets
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from tqdm import tqdm


c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# Hyperparameters
# num_classes = 10
# learning_rate = 0.001
# batch_size = 64
# num_epochs = 5
# optimizer = optim.Adam
# activation_func = nn.ReLU()
# dropout_rate = 0.1
# no_neurons = 128
# criterion = nn.CrossEntropyLoss()
# kernel_size=3
# stride=1
# padding=1
# number_of_filters0=32
# number_of_filters1=32
# length_of_input0=32



class CNN_3_class(nn.Module):
    def __init__(self, num_classes = 10
                    ,kernel_size=3
                    ,stride=1
                    ,padding=1
                    ,number_of_filters0=32
                    ,number_of_filters1=32
                    ,length_of_input0=32
                    ,no_neurons = 128):
        super(CNN_3_class, self).__init__()
        self.conv1 = nn.Conv2d(3, number_of_filters0, kernel_size, stride, padding)
        self.pool1 = nn.MaxPool2d(2)
        length_of_input1 = self.Conv2d_output_size(length_of_input0, kernel_size, stride, padding)/2
        self.conv2 = nn.Conv2d(number_of_filters0, number_of_filters1, kernel_size, stride, padding)
        self.pool2 = nn.MaxPool2d(2)
        length_of_input2 = self.Conv2d_output_size(length_of_input1, kernel_size, stride, padding)/2
        self.fc1 = nn.Linear(int(number_of_filters1*length_of_input2*length_of_input2), no_neurons)
        self.fc2 = nn.Linear(no_neurons, num_classes)
        # parameters
        self.num_classes = num_classes
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.number_of_filters0 = number_of_filters0
        self.number_of_filters1 = number_of_filters1
        self.length_of_input0 = length_of_input0
        self.no_neurons = no_neurons


    def Conv2d_output_size(self, w, k, s, p):
        '''
        w - width of input image
        k - kernel size
        s - stride
        p - padding
        '''
        return (w - k + 2 * p) / s + 1
        
    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        length_of_input1 = self.Conv2d_output_size(self.length_of_input0, self.kernel_size, self.stride, self.padding)/2
        x = self.pool2(torch.relu(self.conv2(x)))
        length_of_input2 = self.Conv2d_output_size(length_of_input1, self.kernel_size, self.stride, self.padding)/2
        x = x.view(-1, int(self.number_of_filters1*length_of_input2*length_of_input2))
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def train_step(self, data, optimizer, criterion):
        x, y = data

        optimizer.zero_grad()

        logits = self(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        accuracy = (logits.argmax(dim=1) == y).float().mean()

        return {'loss': loss, 'accuracy': accuracy}

    def test_step(self, data, criterion):
        x, y = data

        logits = self(x)
        loss = criterion(logits, y)
        accuracy = (logits.argmax(dim=1) == y).float().mean()

        return {'loss': loss, 'accuracy': accuracy}

In [13]:

test_loader = datasets.val_loader
train_loader = datasets.train_loader
augmentation1_loader = datasets.augmentation1_loader
augmentation2_loader = datasets.augmentation2_loader

In [18]:
learning_rate = 0.001
# batch_size = 64 TEGO NIE UZYWAMY A JESLI CHCEMY TO W LOADERZE TRZEBA DODAC
num_epochs = 2
optimizer = optim.Adam
activation_func = nn.ReLU()
# dropout_rate = 0.1
criterion = nn.CrossEntropyLoss()

# Create model, criterion, and optimizer
model = CNN_3_class(num_classes = 10,
                    kernel_size=3,
                    stride=1,
                    padding=1,
                    number_of_filters0=64,
                    number_of_filters1=64,
                    length_of_input0=32,
                    no_neurons = 16)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
epochs = num_epochs
for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    train_losses = []
    train_accuracies = []
    for data in tqdm(train_loader):
        results = model.train_step(data, optimizer, criterion)
        train_losses.append(results['loss'].item())
        train_accuracies.append(results['accuracy'].item())

    # Calculate average training loss and accuracy for the epoch
    avg_train_loss = sum(train_losses) / len(train_losses)
    avg_train_accuracy = sum(train_accuracies) / len(train_accuracies)
    print(f'Train loss: {avg_train_loss:.4f}, Train accuracy: {avg_train_accuracy:.4f}')

    # Test the model
    val_losses = []
    val_accuracies = []
    with torch.no_grad():
        for data in tqdm(test_loader):
            results = model.test_step(data, criterion)
            val_losses.append(results['loss'].item())
            val_accuracies.append(results['accuracy'].item())

    # Calculate average test loss and accuracy for the epoch
    avg_validation_loss = sum(val_losses) / len(val_losses)
    avg_validation_accuracy = sum(val_accuracies) / len(val_accuracies)
    print(f'Validation loss: {avg_validation_loss:.4f}, Validation accuracy: {avg_validation_accuracy:.4f}')


Epoch 1/2


100%|██████████| 1250/1250 [02:30<00:00,  8.31it/s]


Train loss: 1.6507, Train accuracy: 0.3935


100%|██████████| 313/313 [00:49<00:00,  6.30it/s]


Validation loss: 1.4032, Validation accuracy: 0.4810
Epoch 2/2


100%|██████████| 1250/1250 [00:52<00:00, 23.60it/s]


Train loss: 1.3090, Train accuracy: 0.5227


100%|██████████| 313/313 [00:06<00:00, 45.12it/s]

Validation loss: 1.2547, Validation accuracy: 0.5353


### CNN from article about weighted random search

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import datasets
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from tqdm import tqdm

class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=736, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=736, out_channels=508, kernel_size=3, padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(in_channels=508, out_channels=664, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=664, out_channels=916, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv5 = nn.Conv2d(in_channels=916, out_channels=186, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(in_channels=186, out_channels=352, kernel_size=3, padding=1)
        self.linear = nn.Linear(in_features=22528, out_features=1229)
        self.output = nn.Linear(in_features=1229, out_features=num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.maxpool1(x)
        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = self.conv4(x)
        x = nn.functional.relu(x)
        x = self.maxpool2(x)
        x = self.conv5(x)
        x = nn.functional.relu(x)
        x = self.conv6(x)
        x = nn.functional.relu(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        x = nn.functional.relu(x)
        x = self.output(x)
        return x



    def train_step(self, data, optimizer, criterion):
        x, y = data

        optimizer.zero_grad()

        logits = self(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        accuracy = (logits.argmax(dim=1) == y).float().mean()

        return {'loss': loss, 'accuracy': accuracy}

    def test_step(self, data, criterion):
        x, y = data

        logits = self(x)
        loss = criterion(logits, y)
        accuracy = (logits.argmax(dim=1) == y).float().mean()

        return {'loss': loss, 'accuracy': accuracy}